# LangChain

TruLens provides TruChain, a deep integration with LangChain to allow you to inspect and evaluate the internals of your application built using LangChain.

TruChain captures all of the metrics and metadata listed in the [instrumentation overview](./basic_instrumentation). In addition, TruChain instruments the following LangChain classes:

* langchain.chains.base.Chain
* langchain.vectorstores.base.BaseRetriever
* langchain.schema.BaseRetriever
* langchain.llms.base.BaseLLM
* langchain.prompts.base.BasePromptTemplate
* langchain.schema.BaseMemory
* langchain.schema.BaseChatMessageHistory

TruChain also provides async support for Langchain through the `acall` method.

You can find the quickstart available here: [LangChain Quickstart])(../langchain_quickstart)

For more usage examples, check out the [LangChain examples directory](https://github.com/truera/trulens/tree/main/trulens_eval/examples/frameworks/langchain).